In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import sys

sys.path.append('../')
print(sys.path)

['/root/miniconda3/envs/glm-lora/lib/python38.zip', '/root/miniconda3/envs/glm-lora/lib/python3.8', '/root/miniconda3/envs/glm-lora/lib/python3.8/lib-dynload', '', '/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages', '/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/setuptools/_vendor', '../']


In [3]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)


# model = AutoModel.from_pretrained("THUDM/chatglm-6b", load_in_8bit=True, trust_remote_code=True, device_map='auto')
model = AutoModel.from_pretrained("../chatglm-6b", load_in_8bit=True, trust_remote_code=True, device_map='auto')
model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda121.so...


/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /root/miniconda3/envs/glm-lora did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('https'), PosixPath('//u506680-b7d7-4e899b10.cqa1.seetacloud.com'), PosixPath('8443')}
  warn(msg)
/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('Asia/Shanghai')}
  warn(msg)
/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('"en","resolvedLanguage"'), Posi

In [4]:
tokenizer = AutoTokenizer.from_pretrained('../chatglm-6b', trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [5]:
"""
Test before finetuning
"""

from cover_alpaca2jsonl import format_example
import json


instructions = json.load(open("data/alpaca_data.json"))


with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The dtype of attention mask (torch.int64) is not bool


Instruction: Give three tips for staying healthy.
Answer: 1. Eat a balanced diet that includes a variety of fruits, vegetables, whole grains, lean proteins, and healthy fats.
2. Exercise regularly, either through physical activity or simply by getting up and moving around.
3. Get enough sleep each night, as sleep is essential for maintaining good health.
### 1.Answer:
 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule. 


Instruction: What are the three primary colors?
Answer: The three primary colors are red, blue, and yellow. These colors are used in painting, printing, and other visual arts to create colors that can be easily mixed together to create other colors.
### 2.Answer:
 The three primary colors are red, blue, and yellow. 


Instruction: Describe the structure of an atom.
Answer: The structure of an atom is made up of a centr

In [6]:
"""Start finetuning"""

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=8,
    lora_alpha=32, lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)
model.is_parallelizable = False
model.model_parallel = False

In [7]:
import datasets

dataset_path = "data/alpaca/"

dataset = datasets.load_from_disk(dataset_path)

train_num = 500

mini_train_dataset = datasets.Dataset.from_dict(dataset[:train_num])

In [8]:
from transformers import Trainer, HfArgumentParser


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
            [-100] * (seq_len - 1) + ids[(seq_len - 1) :] + [-100] * (longest - ids_l)
        )
        ids = ids + [tokenizer.pad_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
    }

class ModifiedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        return model(
            input_ids=inputs["input_ids"],
            labels=inputs["labels"],
        ).loss

In [9]:
training_args = TrainingArguments(
    "output",
    fp16 =True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 1,
    learning_rate = 1e-4,
    max_steps=1500,
    logging_steps=50,
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
)


trainer = ModifiedTrainer(
    model=model,
    train_dataset=mini_train_dataset,
    args=training_args,
    data_collator=data_collator,
)
trainer.train()

/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,2.425300
100,1.764300
150,1.696800
200,1.604300
250,1.671100
300,1.475800
350,1.513800
400,1.419700
450,1.263700
500,1.443800


/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=1500, training_loss=1.3038308537801107, metrics={'train_runtime': 687.4507, 'train_samples_per_second': 2.182, 'train_steps_per_second': 2.182, 'total_flos': 3697453324468224.0, 'train_loss': 1.3038308537801107, 'epoch': 3.0})

In [10]:
"""Test after finetuning"""

from cover_alpaca2jsonl import format_example
import json


instructions = json.load(open("data/alpaca_data.json"))


with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/root/miniconda3/envs/glm-lora/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Instruction: Give three tips for staying healthy.
Answer: 1. Eat a balanced diet. 
2. Exercise regularly. 
3. Get enough sleep.
### 1.Answer:
 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule. 


Instruction: What are the three primary colors?
Answer: The three primary colors are red, blue, and yellow.
### 2.Answer:
 The three primary colors are red, blue, and yellow. 


Instruction: Describe the structure of an atom.
Answer: An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons. The protons and neutrons are made up of positive and negative charges, while the electrons are negatively charged particles that orbit the nucleus. The protons and neutrons are arranged in a specific configuration, known as the atomic nucleus, which is the basic building block of the atom. The electrons are arranged in a

In [11]:
import os


def save_tunable_parameters(model, path):
    saved_params = {
        k: v.to("cpu")
        for k, v in model.named_parameters()
        if v.requires_grad
    }
    torch.save(saved_params, path)


save_tunable_parameters(model, os.path.join("output", "chatglm-lora.pt"))